<a href="https://colab.research.google.com/github/Zezoo123/DeFi-Arbitrage-and-Prediction-Tool/blob/nn_implementation/neural_networks/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [49]:
folderpath = '/content/drive/MyDrive/BTC/'

df = pd.read_csv(folderpath + '1m.csv')
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2024-09-17 18:17:00,60979.68,61010.00,60979.67,61003.92,8.18716,2024-09-17 18:17:59.999,499422.969558,2118,6.44029,392864.909995,0
1,2024-09-17 18:18:00,61003.92,61017.89,60960.00,60960.00,14.22650,2024-09-17 18:18:59.999,867831.694495,3444,1.20936,73774.472340,0
2,2024-09-17 18:19:00,60960.00,60960.00,60900.23,60927.80,14.63748,2024-09-17 18:19:59.999,891840.587857,3037,9.38786,571997.491349,0
3,2024-09-17 18:20:00,60927.80,60934.00,60852.00,60874.00,11.03523,2024-09-17 18:20:59.999,671870.062854,5103,1.24145,75585.317615,0
4,2024-09-17 18:21:00,60873.99,60940.00,60873.99,60936.00,7.86445,2024-09-17 18:21:59.999,478978.199634,2402,5.52256,336315.909752,0


In [50]:
# Preprocess data
df_processed = df[['Open', 'High', 'Low', 'Close', 'Volume']]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df_processed)
df_processed = pd.DataFrame(scaled_data, columns=df_processed.columns)
df_processed.head()

,Open,High,Low,Close,Volume
0,0.215731,0.211417,0.216855,0.218307,0.007105
1,0.218307,0.212256,0.214758,0.213641,0.012432
2,0.213641,0.206098,0.208385,0.210220,0.012794
3,0.210220,0.203333,0.203242,0.204504,0.009617
4,0.204503,0.203971,0.205586,0.211091,0.006821


In [51]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(data, window_size=5):
  X = []
  y = []
  close_prices = data['Close']
  for i in range(len(data) - window_size):
    X.append(data.iloc[i: i + window_size].values)
    y.append(close_prices[i + window_size]) # Close Value
  return np.array(X), np.array(y)

In [52]:
WINDOW_SIZE = 5

X, y = df_to_X_y(df_processed, window_size=WINDOW_SIZE)
X.shape, y.shape

((43135, 5, 5), (43135,))

In [53]:
#Split the data
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

In [54]:
#Create LSTM model
model = tf.keras.Sequential(([
    tf.keras.layers.InputLayer(shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)  # Predict the 'Close' price
]))

#use adam optimizer
#loss calculated using mean squared error
model.compile(optimizer='adam', loss='mse')

In [55]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.0060 - val_loss: 9.0778e-05
Epoch 2/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 4.4631e-05 - val_loss: 1.0423e-04
Epoch 3/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 4.4232e-05 - val_loss: 5.9099e-05
Epoch 4/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - loss: 4.8060e-05 - val_loss: 1.4603e-04
Epoch 5/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 4.7033e-05 - val_loss: 5.8906e-05
Epoch 6/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 4.5096e-05 - val_loss: 9.6516e-05
Epoch 7/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 4.1631e-05 - val_loss: 6.5340e-05
Epoch 8/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 3.4283e-05 - val_loss: 6.0321e-05
Epoch 9/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 3.2748e-05 - val_loss: 5.7189e-05
Epoch 10/20
1079/1079 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 2.6532e-05 - val_loss: 2.6245e-05
Epoch 11/20
1079/1079 ━━━━━━━━━━━━━━━

In [48]:
#Evaluate
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7280e-04 
Test Loss: 0.00070578494342044
